In [42]:
import pandas as pd
from pathlib import Path
import altair as alt

In [20]:
state_lvl = pd.read_excel('state_level_est.xlsx')

In [21]:
state_lvl.head()

,STATE_NAME,STUDYYEAR,TYPE,INFANT_POPULATION,TODDLER_POPULATION,PRESCHOOL_POPULATION,SCHOOL_AGE_POPULATION,PRICE_Q1_INFANT,PRICE_Q2_INFANT,PRICE_Q3_INFANT,...,AFFORDABILITY_Q4_SCHOOLAGE,AFFORDABILITY_Q5_SCHOOLAGE,MEDIAN_INFANT_PRICE,MEDIAN_TODDLER_PRICE,MEDIAN_PRESCHOOL_PRICE,MEDIAN_SCHOOLAGE_PRICE,PCT_VALID_INFANT,PCT_VALID_TODDLER,PCT_VALID_PRESCHOOL,PCT_VALID_SCHOOLAGE
0,Alabama,2008,Center,119824,122575,63274,490072,0.27,0.44,0.29,...,0.23,0.10,107.45,107.45,95.44,94.10,1.0,1.0,1.0,1.0
1,Alabama,2008,Family,119824,122575,63274,490072,0.24,0.28,0.09,...,0.13,0.21,92.19,92.19,90.65,85.51,1.0,1.0,1.0,1.0
2,Alabama,2009,Center,120656,123426,63714,489742,0.27,0.45,0.28,...,0.22,0.10,109.28,109.28,97.60,95.10,1.0,1.0,1.0,1.0
3,Alabama,2009,Family,120656,123426,63714,489742,0.27,0.12,0.26,...,0.15,0.22,95.10,95.10,93.39,89.11,1.0,1.0,1.0,1.0
4,Alabama,2010,Center,118796,121523,62731,496710,0.27,0.45,0.28,...,0.22,0.11,108.62,108.62,98.10,94.43,1.0,1.0,1.0,1.0


In [52]:
state_lvl[(state_lvl['STUDYYEAR'] == 2022) & (state_lvl['TYPE'] == 'Center')].MEDIAN_INFANT_PRICE.isna().sum()

np.int64(7)

In [56]:
state_lvl[(state_lvl['STUDYYEAR'] == 2022) & (state_lvl['MEDIAN_SCHOOLAGE_PRICE'].isna()) & (state_lvl['TYPE'] == 'Center')]

,STATE_NAME,STUDYYEAR,TYPE,INFANT_POPULATION,TODDLER_POPULATION,PRESCHOOL_POPULATION,SCHOOL_AGE_POPULATION,PRICE_Q1_INFANT,PRICE_Q2_INFANT,PRICE_Q3_INFANT,...,AFFORDABILITY_Q4_SCHOOLAGE,AFFORDABILITY_Q5_SCHOOLAGE,MEDIAN_INFANT_PRICE,MEDIAN_TODDLER_PRICE,MEDIAN_PRESCHOOL_PRICE,MEDIAN_SCHOOLAGE_PRICE,PCT_VALID_INFANT,PCT_VALID_TODDLER,PCT_VALID_PRESCHOOL,PCT_VALID_SCHOOLAGE
118,Arkansas,2022,Center,71079,72711,37534,311329,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,District of Columbia,2022,Center,16277,16650,8595,53565,NaN,NaN,NaN,...,NaN,NaN,490.0,450.6,392.5,NaN,1.0,1.0,1.0,NaN
448,Indiana,2022,Center,160553,164239,84782,700945,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
778,Missouri,2022,Center,141189,144430,74556,609051,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
958,New Mexico,2022,Center,45443,46487,23997,213516,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1168,Pennsylvania,2022,Center,269920,276117,142534,1180320,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1198,Puerto Rico,2022,Center,44248,45264,23366,259429,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,Vermont,2022,Center,11084,11338,5853,52310,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
#state avg multi scatter
def state_avg_costs_df(df):
    #2022 center data
    #eliminate states that don't have data for all four age groups
    #eliminate states where less than 90% of kids are living in counties w data
    clean_df = df[(df['STUDYYEAR'] == 2022) & (df['MEDIAN_INFANT_PRICE'].notna()) & (df['TYPE'] == 'Center') & (df['PCT_VALID_INFANT'] >= .9) & (df['PCT_VALID_TODDLER'] >= .9) & (df['PCT_VALID_PRESCHOOL'] >= .9) & (df['PCT_VALID_SCHOOLAGE'] >= .9)]
    select_columns_df = clean_df[['STATE_NAME', 'MEDIAN_INFANT_PRICE', 'MEDIAN_TODDLER_PRICE', 'MEDIAN_PRESCHOOL_PRICE', 'MEDIAN_SCHOOLAGE_PRICE']]
    
    return select_columns_df

In [113]:
random_census = pd.read_csv('census_mhi_2022.csv')

In [117]:
random_census.dtypes

State    object
MHI       int64
dtype: object

In [120]:
random_census.rename(columns={"State":"STATE_NAME", "MHI":"MHI"})

,STATE_NAME,MHI
0,United States,74755
1,Alabama,59674
2,Alaska,88121
3,Arizona,74568
4,Arkansas,55432
5,California,91551
6,Colorado,89302
7,Connecticut,88429
8,Delaware,82174
9,District of Columbia,101027


In [128]:
def grouped_bar_df(df, census_df):
    df = df.melt(
        id_vars=['STATE_NAME'],
        value_vars=['MEDIAN_INFANT_PRICE', 'MEDIAN_TODDLER_PRICE', 'MEDIAN_PRESCHOOL_PRICE', 'MEDIAN_SCHOOLAGE_PRICE'],
        var_name='age',
        value_name='price'
    )
    df['age'] = df['age'].str.replace('_PRICE', '').str.replace('MEDIAN_', '').str.lower()
    census_df = census_df.rename(columns={"State":"STATE_NAME", "MHI":"MHI"})
    merged = pd.merge(df, census_df, "left", "STATE_NAME")
    merged["adjusted price"] = merged['price'] * 52 / merged['MHI']

    return merged

In [64]:
df = state_lvl
df = df[(df['STUDYYEAR'] == 2022) & (df['MEDIAN_INFANT_PRICE'].notna()) & (df['TYPE'] == 'Center')]
df[df['PCT_VALID_SCHOOLAGE'] >= .9].PCT_VALID_SCHOOLAGE.count()

np.int64(41)

In [122]:
df = state_avg_costs_df(state_lvl)

In [129]:
gdf = grouped_bar_df(df, random_census)

In [130]:
gdf

,STATE_NAME,age,price,MHI,adjusted price
0,Alabama,infant,136.01,59674,0.118519
1,Arizona,infant,270.00,74568,0.188285
2,California,infant,379.21,91551,0.215387
3,Colorado,infant,377.40,89302,0.219758
4,Connecticut,infant,350.00,88429,0.205815
...,...,...,...,...,...
159,Utah,schoolage,163.68,89168,0.095453
160,Virginia,schoolage,155.25,85873,0.094011
161,Washington,schoolage,199.61,91306,0.113681
162,West Virginia,schoolage,146.45,54329,0.140172


In [ ]:
df.melt(
    id_vars=['STATE_NAME'],
    value_vars=['MEDIAN_INFANT_PRICE', 'MEDIAN_TODDLER_PRICE', 'MEDIAN_PRESCHOOL_PRICE', 'MEDIAN_SCHOOLAGE_PRICE'],
    var_name='age',
    value_name='price'
)
df_long['age'] = df_long['age'].str.replace('_price', '')

,STATE_NAME,age,price
0,Alabama,MEDIAN_INFANT_PRICE,136.01
1,Arizona,MEDIAN_INFANT_PRICE,270.00
2,California,MEDIAN_INFANT_PRICE,379.21
3,Colorado,MEDIAN_INFANT_PRICE,377.40
4,Connecticut,MEDIAN_INFANT_PRICE,350.00
...,...,...,...
159,Utah,MEDIAN_SCHOOLAGE_PRICE,163.68
160,Virginia,MEDIAN_SCHOOLAGE_PRICE,155.25
161,Washington,MEDIAN_SCHOOLAGE_PRICE,199.61
162,West Virginia,MEDIAN_SCHOOLAGE_PRICE,146.45


In [134]:
def state_avg_groupedbar(df):
    age_order = ['infant', 'toddler', 'preschool', 'schoolage']
    alt.Chart(df).mark_bar().encode(
        alt.X('age:N', sort=age_order).title('Age Group'),
        alt.Y('price:Q').title('Weekly Cost of Childcare for One Child'),
        color='age:N',
        column='STATE_NAME:N'
    ).show()

In [133]:
def state_avg_norm_groupedbar(df):
        age_order = ['infant', 'toddler', 'preschool', 'schoolage']
        alt.Chart(df).mark_bar().encode(
            alt.X('age:N', sort=age_order).title('Age Group'),
            alt.Y('adjusted price:Q').title('Percentage of Median Household Income'),
            color='age:N',
            column='STATE_NAME:N'
        ).show()

In [132]:
state_avg_norm_groupedbar(gdf)

alt.Chart(...)

In [135]:
state_avg_groupedbar(gdf)

alt.Chart(...)

In [ ]:
def state_avg_splot(df):
    chart = alt.Chart(df)
    
    infant = chart.mark_point(color="purple").encode(
        alt.X('pop_2020').title("State Population, 2020"),
        alt.Y('upper').title('Total Legislative Seats'),
    )

    schoolage = 

(41, 5)

In [ ]:
def state_avg_splot_norm(df):
    

In [22]:
ndcp = pd.read_excel("2022ndcp.xlsx")

In [142]:
il_stuff = ndcp[['STATE_ABBREVIATION', 'COUNTY_NAME', 'COUNTY_FIPS_CODE', 'STUDYYEAR', 'MHI', 'MCSA', 'MCINFANT', 'MCTODDLER', 'MCPRESCHOOL']]

In [143]:
il_stuff = il_stuff[(il_stuff['STATE_ABBREVIATION'] == 'IL') & (il_stuff['STUDYYEAR'] == 2022)]

In [144]:
il_stuff

,STATE_ABBREVIATION,COUNTY_NAME,COUNTY_FIPS_CODE,STUDYYEAR,MHI,MCSA,MCINFANT,MCTODDLER,MCPRESCHOOL
8941,IL,Adams County,17001,2022,63767,30.15,214.12,151.67,151.67
8956,IL,Alexander County,17003,2022,40365,NaN,NaN,NaN,NaN
8971,IL,Bond County,17005,2022,58617,37.00,190.78,198.33,205.00
8986,IL,Boone County,17007,2022,80502,185.73,288.33,221.67,181.67
9001,IL,Brown County,17009,2022,64760,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10396,IL,Whiteside County,17195,2022,62828,43.75,236.33,209.67,175.00
10411,IL,Will County,17197,2022,103678,58.35,304.33,281.67,255.00
10426,IL,Williamson County,17199,2022,60325,28.40,264.12,205.00,171.67
10441,IL,Winnebago County,17201,2022,61738,43.15,281.67,215.00,184.53


In [148]:
il_clean = il_stuff[(il_stuff['MCSA'].notna()) & (il_stuff['MCTODDLER'].notna())]

In [149]:
il_clean

,STATE_ABBREVIATION,COUNTY_NAME,COUNTY_FIPS_CODE,STUDYYEAR,MHI,MCSA,MCINFANT,MCTODDLER,MCPRESCHOOL
8941,IL,Adams County,17001,2022,63767,30.15,214.12,151.67,151.67
8971,IL,Bond County,17005,2022,58617,37.00,190.78,198.33,205.00
8986,IL,Boone County,17007,2022,80502,185.73,288.33,221.67,181.67
9076,IL,Champaign County,17019,2022,61090,51.55,309.47,275.00,239.00
9091,IL,Christian County,17021,2022,56933,41.65,155.45,158.33,141.67
9136,IL,Clinton County,17027,2022,78054,40.90,183.85,170.33,162.33
9151,IL,Coles County,17029,2022,53732,35.25,172.98,158.33,148.33
9166,IL,Cook County,17031,2022,78304,82.60,278.33,220.00,189.17
9211,IL,DeKalb County,17037,2022,68617,70.00,292.73,240.00,235.83
9256,IL,DuPage County,17043,2022,107035,75.00,374.33,280.00,315.83


In [ ]:
state_costs_df = state_avg_costs_df(state_lvl_df)
state_costs_df = pd.merge(state_costs_df, census_mhi_df, "left", "STATE_NAME")
state_costs_df['adj_infant'] = (state_costs_df["MEDIAN_INFANT_PRICE"] * 52) / state_costs_df['MHI']
state_costs_df['adj_schoolage'] = (state_costs_df["MEDIAN_SCHOOLAGE_PRICE"] * 52) / state_costs_df['MHI']

def schoolage_v_infant_heatmap(df):
    alt.Chart(df).mark_rect().encode(
        alt.X('adj_schoolage:Q').bin(maxbins=15),
        alt.Y('adj_infant:Q').bin(maxbins=15),
        alt.Color('count():Q').scale(scheme='greenblue')
    ).show()

schoolage_v_infant_heatmap(state_costs_df)

il_df = clean_il_df(big_22_ndcp_df)
il_df['adj_infant'] = (il_df["MCINFANT"] * 52) / il_df['MHI']
il_df['adj_schoolage'] = (il_df["MCSA"] * 52) / il_df['MHI']

schoolage_v_infant_heatmap(il_df)



In [157]:
il_clean = pd.merge(il_clean, ndcp[['COUNTY_NAME', 'FLFPR_20to64',
                                    'FLFPR_20to64_UNDER6', 'FLFPR_20to64_6to17',
                                    'FLFPR_20to64_UNDER6_6to17']], 'left', 'COUNTY_NAME')

In [158]:
il_clean

,STATE_ABBREVIATION,COUNTY_NAME,COUNTY_FIPS_CODE,STUDYYEAR,MHI,MCSA,MCINFANT,MCTODDLER,MCPRESCHOOL,FLFPR_20to64,FLFPR_20to64_UNDER6,FLFPR_20to64_6to17,FLFPR_20to64_UNDER6_6to17
0,IL,Adams County,17001,2022,63767,30.15,214.12,151.67,151.67,74.4,64.3,83.0,64.2
1,IL,Adams County,17001,2022,63767,30.15,214.12,151.67,151.67,74.1,64.4,81.6,63.2
2,IL,Adams County,17001,2022,63767,30.15,214.12,151.67,151.67,74.3,64.6,81.0,62.7
3,IL,Adams County,17001,2022,63767,30.15,214.12,151.67,151.67,75.0,66.8,80.2,62.1
4,IL,Adams County,17001,2022,63767,30.15,214.12,151.67,151.67,74.8,67.4,79.6,61.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,IL,Winnebago County,17201,2022,61738,43.15,281.67,215.00,184.53,78.6,82.2,80.7,69.3
3536,IL,Winnebago County,17201,2022,61738,43.15,281.67,215.00,184.53,78.7,81.1,82.8,71.5
3537,IL,Winnebago County,17201,2022,61738,43.15,281.67,215.00,184.53,79.5,79.7,85.0,72.3
3538,IL,Winnebago County,17201,2022,61738,43.15,281.67,215.00,184.53,80.3,84.5,84.4,72.2
